## Genero la carga de datos a las tablas:
- dim_barrio.
- dim_codigo_tienda.
- dim_coordenada.
- dim_documentos.
- dim_tienda.
- dim_tipo_documento.
- fact_table

In [1]:
import pandas as pd
import numpy as np
from connection import connect_DDBB

# Me conecto a la base de datos
engine = connect_DDBB()

In [2]:
# En esta celda realizo la lectura del dataset


dataset = '..\DataSet\dataset.xlsx'
df = pd.read_excel(dataset)
    


In [3]:
# Realizo la carga de datos a la tabla dim_tienda

tipo_tienda = df['tipo_tienda'].unique().tolist() #me devuelve los valores unicos del tipo de tienda
dim_tienda = pd.DataFrame({'id_tienda':range(1, len(tipo_tienda)+1)})
dim_tienda['tipo_tienda'] = tipo_tienda
dim_tienda.to_sql('dim_tienda', con=engine, if_exists='append', index=None)
dim_tienda

,id_tienda,tipo_tienda
0,1,Tienda especializada
1,2,Tienda Outlet
2,3,Supermercado
3,4,Hipermercado
4,5,Minimercado
5,6,Tienda Regional


In [4]:
# Realizo la carga de datos a la tabla dim_documentos

num_documentos = df['num_documento_cliente'].unique().tolist() #me devuelve los valores unicos del numero de documento
dim_documentos = pd.DataFrame({'id_documento':range(1, len(num_documentos)+1)})
dim_documentos['num_documento_cliente'] = num_documentos
dim_documentos.to_sql('dim_documentos', con=engine, if_exists='append', index=None)
dim_documentos.head(5)

,id_documento,num_documento_cliente
0,1,6861792605025121524
1,2,1552970090928921643
2,3,-4075904842556875666
3,4,3539339627033221084
4,5,5595554501572542610


In [5]:
# Realizo la carga de datos a la tabla dim_barrio

dim_barrio = df.loc[:,['id_barrio', 'nombre_barrio']]
dim_barrio = dim_barrio.drop_duplicates(subset='id_barrio')
dim_barrio = dim_barrio.sort_values(by='id_barrio')
dim_barrio.to_sql('dim_barrio', con=engine, if_exists='append', index=None)
dim_barrio.head(5)

,id_barrio,nombre_barrio
739,737995769,Julio Rincón
21559,737995777,Los Rubles
20061,737996696,Aguacatal
52675,737996724,Terrón Colorado
84628,737996726,El Vergel


In [6]:
# realizo la carga de datos a la tabla dim_tipo_documento

tipo_documento_cliente = df['tipo_documento_cliente'].unique().tolist() #me devuelve los valores unicos
tipo_documento_cliente = sorted(tipo_documento_cliente)
descripcion = ['cedula de ciudadania', 'tarjeta de identidad', 'cedula de extrangeria', 'pasaporte', 'permiso especial de permanencia']
dim_tipo_documento = pd.DataFrame({'tipo_documento_cliente': tipo_documento_cliente,
                                   'descripcion': descripcion})
dim_tipo_documento.to_sql('dim_tipo_documento', con=engine, if_exists='append', index=None)
dim_tipo_documento


,tipo_documento_cliente,descripcion
0,1,cedula de ciudadania
1,2,tarjeta de identidad
2,3,cedula de extrangeria
3,4,pasaporte
4,9,permiso especial de permanencia


In [7]:
# Realizo la carga de datos a la tabla dim_codigo_tienda

codigo_tienda = df.loc[:,['codigo_tienda', 'id_barrio', 'tipo_tienda']]
codigo_tienda = codigo_tienda.drop_duplicates(subset=['codigo_tienda', 'tipo_tienda']) #elimino filas duplicadas en base a la columna codigo_tienda y tipo_tienda
dim_codigo_tienda = codigo_tienda.merge(dim_tienda, on='tipo_tienda')
dim_codigo_tienda['id_codigo_tienda'] = range(1, len(dim_codigo_tienda['codigo_tienda'])+1)
dim_codigo_tienda.drop(['tipo_tienda'], axis=1, inplace=True)
dim_codigo_tienda = dim_codigo_tienda.reindex(columns=['id_codigo_tienda', 'codigo_tienda', 'id_barrio', 'id_tienda'])
dim_codigo_tienda.to_sql('dim_codigo_tienda', con=engine, if_exists='append', index=None)
dim_codigo_tienda


,id_codigo_tienda,codigo_tienda,id_barrio,id_tienda
0,1,14812028,737998244,1
1,2,14388938,737998896,1
2,3,12370698,737998244,1
3,4,10313583,738000400,1
4,5,14358071,737996734,1
...,...,...,...,...
6011,6012,1207513153,737998264,6
6012,6013,1923624524,737998023,6
6013,6014,1893324100,737999427,6
6014,6015,1420916093,737998935,6


In [8]:
# Realizo la carga de datos a la tabla dim_coordenada

dim_coordenada = df.loc[:,['codigo_tienda', 'latitud_tienda', 'longitud_tienda']]
dim_coordenada = dim_coordenada.drop_duplicates(subset=['codigo_tienda', 'latitud_tienda']) #elimino filas duplicadas en base a la columna codigo_tienda
dim_coordenada['id'] = range(1, len(dim_coordenada['codigo_tienda'])+1)
dim_coordenada.drop(['codigo_tienda'],axis=1, inplace=True)
dim_coordenada['id_codigo_tienda'] = range(1, len(dim_coordenada['id'])+1)
dim_coordenada = dim_coordenada.reindex(columns=['id', 'latitud_tienda', 'longitud_tienda', 'id_codigo_tienda'])
dim_coordenada.to_sql('dim_coordenada', con=engine, if_exists='append', index=None)
dim_coordenada


,id,latitud_tienda,longitud_tienda,id_codigo_tienda
0,1,3.402118,-76.525681,1
1,2,3.468765,-76.484936,2
2,3,3.400743,-76.524839,3
3,4,3.369544,-76.523664,4
4,5,3.419046,-76.497730,5
...,...,...,...,...
91048,6012,3.456726,-76.533991,6012
91062,6013,3.451145,-76.536980,6013
92343,6014,3.422141,-76.543802,6014
92347,6015,3.454859,-76.519350,6015


In [9]:
# En esta celda cargo los datos correspondientes a la table fact_table

table = df.loc[:,['num_documento_cliente', 'tipo_documento_cliente', 'codigo_tienda', 'fecha_compra']]
fact_table = table.merge(dim_codigo_tienda, on='codigo_tienda').merge(dim_documentos, on='num_documento_cliente').merge(dim_tipo_documento, on='tipo_documento_cliente')
fact_table.drop(['num_documento_cliente', 'codigo_tienda','descripcion', 'id_tienda'], axis=1, inplace=True)
fact_table['id_table'] = range(1, len(fact_table['fecha_compra'])+1)
fact_table = fact_table.reindex(columns=['id_table', 'fecha_compra', 'id_codigo_tienda', 'id_documento', 'tipo_documento_cliente'])
fact_table.to_sql('fact_table', con=engine, if_exists='append', index=None)
fact_table


,id_table,fecha_compra,id_codigo_tienda,id_documento,tipo_documento_cliente
0,1,2022-06-15 15:49:11,1,1,1
1,2,2022-08-12 11:15:06,53,1,1
2,3,2022-06-04 23:58:15,70,1,1
3,4,2022-01-06 04:03:05,119,1,1
4,5,2022-12-26 01:23:02,195,1,1
...,...,...,...,...,...
96933,96934,2022-09-08 14:19:08,5590,4341,4
96934,96935,2022-03-18 17:19:40,5611,4341,4
96935,96936,2022-01-05 07:01:50,5625,4341,4
96936,96937,2022-10-20 21:11:03,5404,6348,9


#### Utilizo Python y SQL para responder lo siguiente:
1. Cuáles son las tiendas con compras de al menos 100 clientes diferentes?
2. Cuáles son los 5 barrios donde la mayor cantidad de clientes únicos realizan compras en tienda "Tienda Regional"?

In [10]:
def sql(query):
    '''
    Función que me permite realizar consultas a la base de datos
    '''
    df = pd.read_sql(query, engine)
    return df
    

In [11]:
# Respuesta a la primera pregunta
query = '''
SELECT 	dt.tipo_tienda
	FROM	dim_tienda dt 
JOIN 	dim_codigo_tienda dct 
	ON		dt.id_tienda = dct.id_tienda 
JOIN  	fact_table ft 
	ON		dct.id_codigo_tienda = ft.id_codigo_tienda
JOIN 	dim_documentos dd 
	ON		ft.id_documento = dd.id_documento 
GROUP BY dt.tipo_tienda 
HAVING COUNT(DISTINCT dd.num_documento_cliente) >= 100
'''
df = sql(query)
df

,tipo_tienda
0,Hipermercado
1,Supermercado
2,Tienda especializada
3,Tienda Outlet
4,Tienda Regional


In [12]:
# Respuesta a la segunda pregunta

query = '''
SELECT 	db.nombre_barrio
	FROM 	dim_barrio db 
JOIN	dim_codigo_tienda dct 
	ON		db.id_barrio = dct.id_barrio 
JOIN 	fact_table ft 
	ON		dct.id_codigo_tienda = ft.id_codigo_tienda 
JOIN 	dim_documentos dd 
	ON		ft.id_documento = dd.id_documento 
JOIN 	dim_tienda dt 
	ON		dct.id_tienda = dt.id_tienda 
WHERE dt.tipo_tienda = 'Tienda Regional'
GROUP BY db.nombre_barrio 
ORDER BY count(distinct dd.num_documento_cliente) 
LIMIT 5
'''
df = sql(query)
df

,nombre_barrio
0,El Vergel
1,Jorge Eliecer Gaitan
2,Cataya-El Gran Limonar
3,Champagnat
4,Las Orquídeas
